# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [22]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [3]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [23]:
config = pd.read_csv('config.csv').iloc[0]
user = config['id']
k_value = config['k']

In [24]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

### Read the Data

In [25]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

# below object stores the ids of images seen in a set for quick search
# alongwith a list of images with their timestamps incase its needed at a later stage
class userObj():
    def __init__(self):
        self.sortedItems = set()
        self.list = dict()

userData = dict()                   # userData is the final preprocessed data
users_dict = dict()
images_dict = dict()
processed_data = np.empty((63497, 178788), dtype=bool)

for line in appreciate_data:
  val = line.split(' ')
  user = int(val[0])
  image = int(val[1])

  if user not in users_dict:
    users_dict[user] = len(users_dict)
  user_id = users_dict[user]
  if image not in images_dict:
    images_dict[image] = len(images_dict)
  image_id = images_dict[image]

  processed_data[user_id][image_id] = True


denseMatrix = csr_matrix(processed_data, dtype=float)

In [37]:
from scipy.sparse.linalg import svds

u, s, vt = svds(denseMatrix, k = 100)


In [38]:
# import sys
# print(sys.getsizeof(u))
# print(u)

128
[[-9.59498831e-03  1.49303760e-02 -1.27644394e-02]
 [-9.47341318e-02 -4.89206245e-02 -8.91411477e-02]
 [-3.01043418e-03  3.17078974e-03 -3.11962589e-03]
 ...
 [ 3.67208157e-04  5.26441290e-04 -2.12721568e-04]
 [-1.64862122e-05 -1.13746059e-05 -2.22996848e-06]
 [ 1.08224005e-03 -5.38257670e-04 -3.79884849e-04]]


In [9]:
# svd = TruncatedSVD(n_components=10, n_iter=1, random_state=68)
# compressedData = svd.fit(processed_data)
# print(compressedData)
# compressedData = svd.transform(processed_data)

In [12]:
import sys
print(compressedData[0].shape)

NameError: name 'compressedData' is not defined

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [ ]:
user_likes = dict()

# distance function, written separately so that it can be changed
# if we want to change the logic
def distance(data1, data2):
    dist = 0

    # jaccard distance method
    if len(data1.sortedItems) > len(data2.sortedItems):
        for item in data2.sortedItems:
            if item in data1.sortedItems:
                dist += 1
    else:
        for item in data1.sortedItems:
            if item in data2.sortedItems:
                dist += 1
    
    # dist = dist/(len(data1.sortedItems) + len(data2.sortedItems) - dist)
    
    return dist

### Use KNN after Collaborative Filtering to find nearest neighbors

In [ ]:
# your code here
def neighbors(user,k_value):
    global userData
    """ returns an iterable object (like list or generator) """
    allDist = []
    for otherUser in userData:
        if otherUser != user:
            allDist.append([distance(userData[otherUser], userData[user]), otherUser])
    allDist.sort()
    
    # creating a list of movies
    newList = []
    for i in range(len(allDist) - 1, max(len(allDist) - k_value - 1, -1), -1):
        newList.append(allDist[i][1])
    
    return newList
    

### Open the output file to write all the lines to the file

In [ ]:
outFile = open(config['output_file'], 'w')
listOfKNN = neighbors(user,k_value)
for n_user in listOfKNN:
    user_id = n_user
    for item_id in userData[user_id].sortedItems:
        if item_id not in userData[user].sortedItems:
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()